# Budget Line Items Analysis
วิเคราะห์ข้อมูลจากตาราง `budget_line_items`

In [ ]:
import sys
print(sys.executable)

: 

In [3]:
import pandas as pd
import mysql.connector
from db_config import get_db_config

# เชื่อมต่อฐานข้อมูล
config = get_db_config()
conn = mysql.connector.connect(
    host=config['host'],
    user=config['user'],
    password=config['password'],
    database=config['database'],
    port=config['port']
)

print("✅ Connected to database successfully!")

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# ดึงข้อมูลทั้งหมดจาก budget_line_items พร้อม JOIN ตารางที่เกี่ยวข้อง
query = """
SELECT 
    bli.id,
    bli.fiscal_year,
    bli.division_id,
    o.name_th as division_name,
    bli.plan_id,
    p.name_th as plan_name,
    bli.project_id,
    proj.name_th as project_name,
    bli.activity_id,
    act.name_th as activity_name,
    bli.allocated_pba,
    bli.allocated_received,
    bli.disbursed
FROM budget_line_items bli
LEFT JOIN organizations o ON bli.division_id = o.id
LEFT JOIN plans p ON bli.plan_id = p.id
LEFT JOIN projects proj ON bli.project_id = proj.id
LEFT JOIN activities act ON bli.activity_id = act.id
ORDER BY bli.fiscal_year, bli.division_id, bli.plan_id
"""

df = pd.read_sql(query, conn)
print(f"📊 Total rows: {len(df)}")
df.head(10)

In [ ]:
# ข้อมูลทั่วไป
df.info()

In [ ]:
# สรุปตามกอง (Division)
print("=== Summary by Division ===")
division_summary = df.groupby(['division_id', 'division_name']).agg({
    'id': 'count',
    'plan_id': 'nunique'
}).rename(columns={'id': 'total_rows', 'plan_id': 'distinct_plans'})
division_summary

In [ ]:
# สรุปตามแผนงาน (Plan)
print("=== Summary by Plan ===")
plan_summary = df.groupby(['plan_id', 'plan_name']).agg({
    'id': 'count',
    'division_id': 'nunique'
}).rename(columns={'id': 'total_rows', 'division_id': 'distinct_divisions'})
plan_summary

In [ ]:
# ข้อมูลที่ division_id = NULL (ยังไม่ได้ assign กอง)
print("=== Records with division_id = NULL ===")
null_div = df[df['division_id'].isna()]
print(f"Total rows: {len(null_div)}")
if len(null_div) > 0:
    null_div.groupby('plan_name').size().sort_values(ascending=False)

In [ ]:
# ข้อมูลของ division_id = 3 (กองบริหารทรัพยากรบุคคล)
print("=== Records for division_id = 3 (กองบริหารทรัพยากรบุคคล) ===")
div3 = df[df['division_id'] == 3]
print(f"Total rows: {len(div3)}")
if len(div3) > 0:
    div3.groupby('plan_name').size().sort_values(ascending=False)

In [ ]:
# ดูข้อมูลของ division_id = 3 ทั้งหมด
div3

In [ ]:
# ปิดการเชื่อมต่อ
conn.close()
print("✅ Connection closed")